In [1]:
import jax; jax.config.update('jax_platform_name', 'cpu')
import time
from pyscf import df as pyscf_df
from pyscfad import gto, dft, scf, df
from pyscfad.gw import rpa, sigma
from pyscf.geomopt.berny_solver import optimize, to_berny_geom
from berny import Berny, geomlib
import warnings
warnings.simplefilter("ignore")

In [2]:
def energy_rpa(mol, with_df):
    mf = dft.RKS(mol)
    mf.xc = 'pbe'
    mf.kernel(dm0=None)

    mymp = rpa.RPA(mf)
    mymp.with_df = with_df
    mymp.kernel()
    return mymp.e_tot

In [3]:
def energy_sigma(mol, with_df):
    mf = dft.RKS(mol)
    mf.xc = 'pbe'
    mf.kernel(dm0=None)

    mymp = sigma.SIGMA(mf)
    mymp.with_df = with_df
    mymp.kernel()
    return mymp.e_tot

In [4]:
def solver(geom, val_and_grad, basis):
    mol = gto.Mole()
    mol.verbose = 0
    mol.atom = geom
    mol.basis = basis
    mol.build(trace_exp=False, trace_ctr_coeff=False)

    auxbasis = pyscf_df.addons.make_auxbasis(mol, mp2fit=True)
    auxmol = df.addons.make_auxmol(mol, auxbasis)
    with_df = df.DF(mol, auxmol=auxmol)

    e_tot, jac = val_and_grad(mol, with_df)

    return e_tot, jac[0].coords + jac[1].mol.coords + jac[1].auxmol.coords

In [5]:
def get_geom(geom_str):
    mol_ = gto.Mole()
    mol_.build(atom = geom_str, basis = 'sto-3g')
    geom = to_berny_geom(mol_)
    return geom

In [6]:
def optimize_geom(geom_, basis, method, verbose):
    ts = time.time()
    print('starting geometry:', geom_)
    print('basis:', basis)
    print('method:', method)

    optimizer = Berny(get_geom(geom_))

    if method is 'RPA':
        val_and_grad = jax.value_and_grad(energy_rpa, (0,1))
    elif method is 'SIGMA':
        val_and_grad = jax.value_and_grad(energy_sigma, (0,1))
    else:
        sys.exit("unknown method")

    print('\nTime before optimization:', time.time()-ts, flush=True)

    print('')
    for iter_, geom in enumerate(optimizer):
        energy, gradients = solver(list(geom), val_and_grad, basis=basis)
        optimizer.send((energy, gradients))
        print(f'iter={iter_+1}   energy={energy:.10f}   elapsed time={time.time()-ts:.2f} seconds', flush=True)
        if verbose:
            print('\nGeometry:')
            print(geom.coords, flush=True)
            print('\nGradients:')
            print(gradients)
            print('')

    print('\nOptimized feometry:')
    print(geom.coords)

In [7]:
optimize_geom(geom_ = '''H 0 0 0; H 0 0 1.''', 
              basis = 'augccpvdz',
              method = 'RPA',
              verbose = False)

starting geometry: H 0 0 0; H 0 0 1.
basis: augccpvdz
method: RPA

Time before optimization: 0.07403993606567383

iter=1   energy=-1.1727842152   elapsed time=9.56 seconds
iter=2   energy=-1.1883282829   elapsed time=15.63 seconds
iter=3   energy=-1.1804935242   elapsed time=21.42 seconds
iter=4   energy=-1.1941174117   elapsed time=27.02 seconds
iter=5   energy=-1.1945424926   elapsed time=31.70 seconds
iter=6   energy=-1.1946426938   elapsed time=37.75 seconds
iter=7   energy=-1.1946435932   elapsed time=41.97 seconds

Optimized feometry:
[[0.         0.         0.11647374]
 [0.         0.         0.88352626]]


In [8]:
optimize_geom(geom_ = '''H 0 0 0; F 0 0 1.''', 
              basis = 'ccpvtz',
              method = 'RPA',
              verbose = False)

starting geometry: H 0 0 0; F 0 0 1.
basis: ccpvtz
method: RPA

Time before optimization: 0.15929722785949707

iter=1   energy=-100.4683497045   elapsed time=17.04 seconds
iter=2   energy=-100.4739767110   elapsed time=29.37 seconds
iter=3   energy=-100.4740075561   elapsed time=41.48 seconds
iter=4   energy=-100.4740114044   elapsed time=53.17 seconds

Optimized feometry:
[[0.         0.         0.03945238]
 [0.         0.         0.96054762]]


In [9]:
optimize_geom(geom_ = '''N 0 0 0; N 0 0 1.''', 
              basis = 'ccpvtz',
              method = 'RPA',
              verbose = False)

starting geometry: N 0 0 0; N 0 0 1.
basis: ccpvtz
method: RPA

Time before optimization: 0.18834424018859863

iter=1   energy=-109.5330225310   elapsed time=24.44 seconds
iter=2   energy=-109.5631064529   elapsed time=45.07 seconds
iter=3   energy=-109.5689238132   elapsed time=65.20 seconds
iter=4   energy=-109.5697433611   elapsed time=84.16 seconds
iter=5   energy=-109.5697909329   elapsed time=102.52 seconds
iter=6   energy=-109.5697911687   elapsed time=121.32 seconds

Optimized feometry:
[[ 0.          0.         -0.05224153]
 [ 0.          0.          1.05224153]]


In [10]:
optimize_geom(geom_ = '''H 0 0 0; H 0 0 1.''', 
              basis = 'augccpvdz',
              method = 'SIGMA',
              verbose = False)

starting geometry: H 0 0 0; H 0 0 1.
basis: augccpvdz
method: SIGMA

Time before optimization: 0.21901154518127441

iter=1   energy=-1.1159028374   elapsed time=54.51 seconds
iter=2   energy=-1.1394035182   elapsed time=108.74 seconds
iter=3   energy=-1.1279648274   elapsed time=160.04 seconds
iter=4   energy=-1.1461803189   elapsed time=212.15 seconds
iter=5   energy=-1.1467156448   elapsed time=266.54 seconds
iter=6   energy=-1.1468317449   elapsed time=319.52 seconds
iter=7   energy=-1.1468327864   elapsed time=371.71 seconds

Optimized feometry:
[[0.         0.         0.12968564]
 [0.         0.         0.87031436]]


In [11]:
optimize_geom(geom_ = '''N 0 0 0; N 0 0 1.''', 
              basis = 'ccpvtz',
              method = 'SIGMA',
              verbose = False)

starting geometry: N 0 0 0; N 0 0 1.
basis: ccpvtz
method: SIGMA

Time before optimization: 0.2563936710357666

iter=1   energy=-109.0040381504   elapsed time=325.51 seconds
iter=2   energy=-108.9898927226   elapsed time=656.98 seconds
iter=3   energy=-109.0182369432   elapsed time=971.81 seconds
iter=4   energy=-109.0196204635   elapsed time=1287.50 seconds
iter=5   energy=-109.0196711596   elapsed time=1611.06 seconds
iter=6   energy=-109.0196713893   elapsed time=1935.87 seconds

Optimized feometry:
[[ 0.          0.         -0.03011028]
 [ 0.          0.          1.03011028]]
